# 01 Time Series Basics

Resampling, lags, rolling windows, and leakage intuition.


## Table of Contents
- [Toy series](#toy-series)
- [Resampling](#resampling)
- [Lag and rolling features](#lag-and-rolling-features)
- [Leakage demo](#leakage-demo)
- [Checkpoint (Self-Check)](#checkpoint-self-check)
- [Solutions (Reference)](#solutions-reference)


## Why This Notebook Matters
Foundations notebooks build the intuition that prevents the most common mistakes in economic ML:
- leaking future information,
- evaluating with the wrong split strategy,
- over-interpreting coefficients.


## What You Will Produce
- (no file output; learning/analysis notebook)

## Success Criteria
- You can explain what you built and why each step exists.
- You can run your work end-to-end without undefined variables.

## Common Pitfalls
- Running cells top-to-bottom without reading the instructions.
- Leaving `...` placeholders in code cells.
- Using random splits on time series.
- Assuming correlation implies causation.

## Matching Guide
- `docs/guides/00_foundations/01_time_series_basics.md`



## How To Use This Notebook
- This notebook is hands-on. Most code cells are incomplete on purpose.
- Complete each TODO, then run the cell.
- Use the matching guide (`docs/guides/00_foundations/01_time_series_basics.md`) for deep explanations and alternative examples.
- Write short interpretation notes as you go (what changed, why it matters).



<a id="environment-bootstrap"></a>
## Environment Bootstrap
Run this cell first. It makes the repo importable and defines common directories.



In [ ]:
from __future__ import annotations

from pathlib import Path
import sys


def find_repo_root(start: Path) -> Path:
    p = start
    for _ in range(8):
        if (p / 'src').exists() and (p / 'docs').exists():
            return p
        p = p.parent
    raise RuntimeError('Could not find repo root. Start Jupyter from the repo root.')


PROJECT_ROOT = find_repo_root(Path.cwd())
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

DATA_DIR = PROJECT_ROOT / 'data'
RAW_DIR = DATA_DIR / 'raw'
PROCESSED_DIR = DATA_DIR / 'processed'
SAMPLE_DIR = DATA_DIR / 'sample'

PROJECT_ROOT



## Concept
This notebook is about *timing*.

Before you touch real economic data, you will practice on a toy series where you can see mistakes clearly.
The same mistakes (especially leakage) show up later when you build recession predictors.



## Primer: pandas Time Series Essentials

### DatetimeIndex
Most time series work in pandas assumes your DataFrame is indexed by time.

```python
import pandas as pd

# idx = pd.to_datetime(df['date'])
# df = df.set_index(idx).sort_index()
# assert isinstance(df.index, pd.DatetimeIndex)
```

If you see weird behavior (resample errors, merges not aligning), check:
- `df.index.dtype`
- `df.index.min(), df.index.max()`
- `df.index.is_monotonic_increasing`

### Resampling
> **Goal:** convert one frequency to another.

Common examples:
- daily -> month-end
- monthly -> quarter-end

```python
# month-end last value
# df_me_last = df.resample('ME').last()

# month-end mean
# df_me_mean = df.resample('ME').mean()

# quarter-end mean
# df_q_mean = df.resample('QE').mean()
```

**Interpretation matters.** For economic series:
- using `.last()` treats the end-of-period value as “the period’s value”
- using `.mean()` treats the period average as “the period’s value”

Neither is universally correct; you should choose based on measurement and use case.

### Alignment and merging
When joining multiple time series, you need to ensure they share:
- the same index type (`DatetimeIndex`)
- the same frequency convention (month-end vs month-start; quarter-end vs quarter-start)

```python
# Example: join two series and inspect missingness
# df = df1.join(df2, how='outer').sort_index()
# print(df.isna().sum())
```

### Lags and rolling windows
> **Lag:** use past values as features.

```python
# lag 1 period
# df['x_lag1'] = df['x'].shift(1)

# rolling mean (past-only)
# df['x_roll12'] = df['x'].rolling(12).mean()
```

### Common gotchas
- `shift(-1)` uses the future (leakage for forecasting).
- `rolling(..., center=True)` uses future values.
- Always `dropna()` after creating lags/rolls to get clean modeling rows.

One more gotcha:
- If you resample daily -> monthly and then create lags, your lag is “one month” (not one day). Lags are measured in the current index frequency.


<a id="toy-series"></a>
## Toy series

### Goal
Create a daily time series with a trend + noise.

### Why this matters
Real macro indicators are noisy. You want to learn how resampling and rolling windows change the information your model sees.

### Your Turn (1): Build an index + values



In [ ]:
import numpy as np
import pandas as pd

# TODO: Create a daily DatetimeIndex from 2010-01-01 for ~5 years.
# Hint: pd.date_range('2010-01-01', periods=..., freq='D')
idx = ...

# TODO: Create a signal with a slow trend + seasonal-ish wiggle + noise.
# Hint: np.linspace + np.sin + rng.normal
rng = np.random.default_rng(0)
trend = ...
season = ...
noise = ...

y = pd.Series(trend + season + noise, index=idx, name='y')
y.head()



### Your Turn (2): Inspect and visualize


In [ ]:
import matplotlib.pyplot as plt

# TODO: Print basic summary stats
# TODO: Plot the time series
...



### Checkpoint (toy series sanity)


In [ ]:
# TODO: Confirm the index is daily and sorted
assert isinstance(y.index, pd.DatetimeIndex)
assert y.index.is_monotonic_increasing
assert y.shape[0] > 1000
assert y.isna().sum() == 0
...



<a id="resampling"></a>
## Resampling

### Goal
Convert daily data to monthly data in two different ways and compare them.

### Why this matters
When you resample, you are choosing a *measurement definition*:
- month-end value (`last`) vs
- monthly average (`mean`).

These can lead to different model conclusions.



### Your Turn (1): Month-end series (mean vs last)


In [ ]:
# TODO: Build two monthly series:
# - y_me_last: month-end value
# - y_me_mean: monthly average

# Hint: y.resample('ME').last() and y.resample('ME').mean()
y_me_last = ...
y_me_mean = ...

y_me_last.head(), y_me_mean.head()



### Your Turn (2): Compare visually and numerically


In [ ]:
# TODO: Put them in one DataFrame and compare.
# - plot both
# - compute correlation
# - compute their difference distribution
df_m = ...
...



### Checkpoint (resampling sanity)


In [ ]:
# TODO: Assert the resampled index is month-end
assert y_me_last.index.freqstr in {'ME', 'M'} or y_me_last.index.inferred_freq in {'M', 'ME'}
assert y_me_last.shape == y_me_mean.shape
...



<a id="lag-and-rolling-features"></a>
## Lag and rolling features

### Goal
Create features that only use the past: lags and rolling windows.

### Why this matters
Most macro prediction is built from lagged indicators (what you knew last month/quarter) and summaries of recent history.



### Your Turn (1): Lags


In [ ]:
# TODO: Create lag features from the DAILY series y:
# - y_lag1: yesterday
# - y_lag7: one week ago
df_feat = pd.DataFrame({'y': y})
df_feat['y_lag1'] = ...
df_feat['y_lag7'] = ...
df_feat.head(10)



### Your Turn (2): Rolling features


In [ ]:
# TODO: Create rolling mean and rolling std features (past-only!)
# Example: 14-day rolling mean and std
df_feat['y_roll14_mean'] = ...
df_feat['y_roll14_std'] = ...

# TODO: drop rows with NaNs created by lags/rolling
df_feat_clean = ...
df_feat_clean.head()



### Checkpoint (feature availability)


In [ ]:
# TODO: Confirm you did not accidentally leak the future.
# Hint: check that lagged columns align the way you expect.
# Example: df_feat.loc[t, 'y_lag1'] should equal df_feat.loc[t - 1 day, 'y']
...



<a id="leakage-demo"></a>
## Leakage demo

### Goal
Experience how leakage can make results look incredible but meaningless.

### What you'll do
1) define a 1-step-ahead prediction task
2) build one legitimate feature set and one leaky feature set
3) compare random split vs time split



### Your Turn (1): Build the prediction dataset


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Predict tomorrow's value
# TODO: target = y.shift(-1)
target = ...

# Legit features: past-only
X_ok = df_feat_clean[['y_lag1', 'y_lag7', 'y_roll14_mean', 'y_roll14_std']].copy()

# LEAKY feature: tomorrow's value (do not do this in real work)
# TODO: X_leak should include a column that equals the target (or uses shift(-1))
X_leak = ...

# Align and drop missing rows
df_model = pd.DataFrame({'target': target}).join(X_ok).dropna()
X_ok = df_model[X_ok.columns]
y_arr = df_model['target']

# TODO: Align X_leak to the same rows as y_arr
...



### Your Turn (2): Random split vs time split


In [ ]:
# Random split (WRONG for time series)
X_tr, X_te, y_tr, y_te = train_test_split(X_ok, y_arr, test_size=0.2, shuffle=True, random_state=0)
m = LinearRegression().fit(X_tr, y_tr)
rmse_rand = mean_squared_error(y_te, m.predict(X_te), squared=False)

# Time split (RIGHT for time series)
split = int(len(X_ok) * 0.8)
X_tr2, X_te2 = X_ok.iloc[:split], X_ok.iloc[split:]
y_tr2, y_te2 = y_arr.iloc[:split], y_arr.iloc[split:]
m2 = LinearRegression().fit(X_tr2, y_tr2)
rmse_time = mean_squared_error(y_te2, m2.predict(X_te2), squared=False)

rmse_rand, rmse_time



### Your Turn (3): Show how the leaky feature 'cheats'


In [ ]:
# TODO: Repeat the evaluation above using X_leak.
# What happens to the test RMSE?
# Write 3-5 sentences explaining why this result is meaningless.
...



<a id="checkpoint-self-check"></a>
## Checkpoint (Self-Check)
Run a few asserts and write 2-3 sentences summarizing what you verified.



In [ ]:
# TODO: Run a quick sanity check on any DataFrame/Series you created in this notebook.
# Example (adjust variable names):
# assert df.index.is_monotonic_increasing
# assert df.isna().sum().sum() == 0
#
# TODO: Write 2-3 sentences:
# - What would leakage look like in YOUR code?
# - How would you detect it?
...



## Extensions (Optional)
- Try one additional variant beyond the main path (different features, different split, different model).
- Write down what improved, what got worse, and your hypothesis for why.



## Reflection
- What did you assume implicitly (about timing, availability, stationarity, or costs)?
- If you had to ship this model, what would you monitor?



<a id="solutions-reference"></a>
## Solutions (Reference)

Try the TODOs first. Use these only to unblock yourself or to compare approaches.

<details><summary>Solution: Toy series</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 01_time_series_basics — Toy series
import numpy as np
import pandas as pd

rng = np.random.default_rng(0)
idx = pd.date_range('2020-01-01', periods=365, freq='D')
y = pd.Series(0.01*np.arange(len(idx)) + rng.normal(0, 1, len(idx)), index=idx, name='y')
y.head()
```

</details>

<details><summary>Solution: Resampling</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 01_time_series_basics — Resampling
monthly_mean = y.resample('ME').mean()
monthly_last = y.resample('ME').last()
monthly_mean.head(), monthly_last.head()
```

</details>

<details><summary>Solution: Lag and rolling features</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 01_time_series_basics — Lag and rolling features
df = y.to_frame()
df['y_lag1'] = df['y'].shift(1)
df['y_lag7'] = df['y'].shift(7)
df['y_roll14_mean'] = df['y'].rolling(14).mean()
df = df.dropna()
df.head()
```

</details>

<details><summary>Solution: Leakage demo</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 01_time_series_basics — Leakage demo
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

tmp = df.copy()
tmp['target_tomorrow'] = tmp['y'].shift(-1)
tmp = tmp.dropna()

X = tmp[['y_lag1', 'y_lag7', 'y_roll14_mean']]
y_t = tmp['target_tomorrow']

# Random split (not recommended for time series)
X_tr, X_te, y_tr, y_te = train_test_split(X, y_t, test_size=0.2, shuffle=True, random_state=0)
m = LinearRegression().fit(X_tr, y_tr)
rmse_rand = mean_squared_error(y_te, m.predict(X_te), squared=False)

# Time split (recommended)
split = int(len(tmp) * 0.8)
m2 = LinearRegression().fit(X.iloc[:split], y_t.iloc[:split])
rmse_time = mean_squared_error(y_t.iloc[split:], m2.predict(X.iloc[split:]), squared=False)

rmse_rand, rmse_time
```

</details>

